# Regression model for IBNR estimates

In [63]:
import itertools
import pandas as pd
import re
import math

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [67]:
def columnas(valores,variable):
    y = [re.findall("\\d+", j)[0] for j in valores]
    y = [int(i) for i in y]
    todas = list(set(y))
    df = pd.DataFrame()
    df[f"y_{variable}"] = y
    for k in todas:
        #print(k)
        df[f"{variable}_{k}"] = ([1 if k == j else 0 for j in y])
    return df

def matrix_X(df_triangulo):
    k = len(df_triangulo.columns)
    alpha = [f'a_{i}' for i in range(1,k+1)]
    mu    = [f'u_{i}' for i in range(1,k+1)]
    lists = [alpha, mu]
    df    = pd.DataFrame(list(itertools.product(*lists)), columns=['a', 'u'])

    alpha    = columnas(valores  = df.a, variable = 'a')
    mu       = columnas(valores=df.u, variable = 'u')
    df_col= pd.concat([alpha, mu], axis=1)


    df_col['y_a'] = df_col['y_a'].astype(str) + df_col['y_u'].astype(str) 
    df_col['y_a'] = [int(i) for i in df_col['y_a']]
    df_col = df_col.drop(['y_u', 'u_1'], axis=1)
    df_col['a_1'] = 1
    df_col.rename(columns={'a_1': 'b0'}, inplace=True)
    df_col.rename(columns={'y_a': 'y_ii'}, inplace=True)
    #df_col = df_col.drop(['y_ii'], axis=1)
    return df_col


def matrix_y(df_triangulo):
    k = len(df_triangulo.columns)
    d0 = pd.DataFrame()
    for i in range(k):
        for j in range(k):
            d1 = pd.DataFrame({'y_ii': [int(f'{i+1}{j+1}')], 'Y': [math.log(df_triangulo.iloc[i, j])]})
            d0 = pd.concat([d0, d1], axis=0)
    return d0

def triangulo(df, grcode, entreno):
    
    if entreno:
        df_trinagulo = df[(df['GRCODE']== grcode ) & (df['DevelopmentYear']<=1997)].copy()
    else: 
        df_trinagulo = df[df['GRCODE']== grcode].copy()
        
    df_g         = df_trinagulo.groupby(["AccidentYear", "DevelopmentLag"]).agg({'IncurLoss_B': ['max']})
    df_g.columns = ['Pagos']
    df_g         = df_g.reset_index()
    pivot_data   = df_g.pivot(index='AccidentYear',columns='DevelopmentLag',values='Pagos').reset_index()
    pivot_data   = pivot_data.drop('AccidentYear', axis=1).cumsum(axis=1)
    
    return pivot_data


In [68]:
input = pd.read_csv(os.path.normpath(os.getcwd() + os.sep + os.pardir)+"/data/ppauto_pos.csv")

input = input[input.DevelopmentYear <= 1997]

cleaning_cond = np.array(['Adriatic Ins Co', 'Aegis Grp', 'Agency Ins Co Of MD Inc',
       'Allegheny Cas Co', 'American Modern Ins Grp Inc',
       'Armed Forces Ins Exchange', 'Auto Club South Ins Co',
       'Baltica-Skandinavia Rein Co Of Amer', 'Bancinsure Inc',
       'Bell United Ins Co', 'Century-Natl Ins Co', 'Co-Operative Ins Co',
       'Consumers Ins Usa Inc', 'Cornerstone Natl Ins Co',
       'Federated Natl Ins Co', 'First Amer Ins Co',
       'Florists Mut Ins Grp', 'Harbor Ins Co', 'Homestead Ins Co',
       'Inland Mut Ins Co', 'Interstate Auto Ins Co Inc', 'Lancer Ins Co',
       'Lumber Ins Cos', 'Manhattan Re Ins Co', 'Mennonite Mut Ins Co',
       'Middle States Ins Co Inc', 'National Automotive Ins',
       'Nevada General Ins Co', 'New Jersey Citizens United Rcp Exch',
       'Nichido Fire & Marine Ins Co Ltd', 'Northwest Gf Mut Ins Co',
       'Ocean Harbor Cas Ins Co', 'Overseas Partners Us Reins Co',
       'Pacific Ind Ins Co', 'Pacific Pioneer Ins Co',
       'Pacific Specialty Ins Co', 'Penn Miller Grp',
       'Pennsylvania Mfg Asn Ins Co', 'Pioneer State Mut Ins Co',
       'Protective Ins Grp', 'San Antonio Reins Co',
       'Seminole Cas Ins Co', 'Southern Group Ind Inc',
       'Southern Mut Ins Co', 'Southland Lloyds Ins Co', 'Star Ins Grp',
       'Sterling Ins Co', 'Usauto Ins Co', 'Vanliner Ins Co',
       'Wea Prop & Cas Ins Co', 'Wellington Ins Co', 'State Farm Mut Grp', 'United Services Automobile Asn Grp',
       'US Lloyds Ins Co', 'Toa-Re Ins Co Of Amer', 'FL Farm Bureau Grp'])

input = input[~input.GRNAME.isin(cleaning_cond)]

array([   43,   266,   353,   388,   460,   620,   671,   692,   715,
         965,  1066,  1090,  1538,  1716,  2143,  2208,  2259,  3240,
        5185,  5320,  5690,  6807,  6947,  7080,  8427,  8559, 10022,
       10049, 10308, 10790, 11037, 11126, 11231, 13420, 13439, 13501,
       13528, 13587, 13595, 13641, 13889, 13943, 14044, 14176, 14257,
       14311, 14370, 14443, 15024, 15199, 15393, 15407, 15660, 15997,
       16373, 16799, 17884, 18163, 18791, 19119, 19780, 23574, 23663,
       23876, 25275, 25755, 26808, 27022, 27065, 27499, 27766, 29297,
       29378, 29440, 31062, 31550, 32387, 32743, 33499, 34509, 34525,
       34592, 34606, 35408, 38997, 40568, 42552, 42749, 42846, 43494])

In [81]:
df_data        = input #pd.read_csv('medmal_pos.csv')
df_trg_entreno = triangulo(df_data, grcode=43, entreno=True)
df_trg_prueba  = triangulo(df_data, grcode=43, entreno=False)
df_trg_entreno

DevelopmentLag,1,2,3,4,5,6,7,8,9,10
0,607.0,1254.0,1836.0,2434.0,3048.0,3663.0,4278.0,4892.0,5506.0,6120.0
1,2254.0,5113.0,8092.0,10856.0,13682.0,16699.0,19689.0,22667.0,25645.0,NaN
2,5843.0,13267.0,21574.0,30245.0,39311.0,48237.0,57004.0,65769.0,NaN,NaN
3,11422.0,27515.0,46163.0,65258.0,83911.0,102380.0,120787.0,NaN,NaN,NaN
4,19933.0,44095.0,72834.0,101163.0,129234.0,156956.0,NaN,NaN,NaN,NaN
5,24604.0,56734.0,90309.0,123078.0,156800.0,NaN,NaN,NaN,NaN,NaN
6,40735.0,84679.0,127190.0,168902.0,NaN,NaN,NaN,NaN,NaN,NaN
7,43064.0,86769.0,129678.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,41837.0,83141.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44436.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
Y = matrix_y(df_trg_entreno)
X = matrix_X(df_trg_entreno)

Y_X          = pd.merge(Y, X, on='y_ii', how='inner')
data_entreno = Y_X[Y_X['Y'].notna()]
data_entreno = data_entreno.drop(['y_ii'], axis=1)
data_entreno.head()

,Y,b0,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,u_2,u_3,u_4,u_5,u_6,u_7,u_8,u_9,u_10
0,6.408529,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7.134094,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,7.515345,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,7.797291,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,8.022241,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [92]:
Y_prueba      = matrix_y(df_trg_prueba)                   
x_prueba      = Y_X[Y_X['Y'].notna()].drop(['Y'], axis=1)
data_prueba_  = pd.merge(Y_prueba, x_prueba, on='y_ii', how='inner')
data_prueba=data_prueba_.drop(['y_ii'], axis=1)
y_ii = data_prueba_['y_ii']


In [93]:
x_entreno = data_entreno.drop('Y', axis=1)  # Features
y_entreno = data_entreno['Y']  # Target variable
x_prueba  = data_prueba.drop('Y', axis=1)  # Features
y_prueba  = data_prueba['Y']  # Target variable


model = LinearRegression()
model.fit(x_entreno, y_entreno)

y_pred = model.predict(x_prueba)
mse = mean_squared_error(y_prueba, y_pred)   # COnsiderar que se debe aplicar la exponencial a los rsultados
mse

0.002230701729399499

In [94]:
y_pred

array([ 6.25925584,  7.03618955,  7.49424014,  7.81488628,  8.0684867 ,
        8.26495225,  8.4245614 ,  8.54791297,  8.64324297,  8.71931738,
        7.73846748,  8.51540119,  8.97345178,  9.29409792,  9.54769834,
        9.74416389,  9.90377304, 10.02712461, 10.12245461,  8.75423106,
        9.53116477,  9.98921536, 10.3098615 , 10.56346192, 10.75992747,
       10.91953662, 11.04288819,  9.48854033, 10.26547404, 10.72352463,
       11.04417077, 11.29777119, 11.49423674, 11.65384589,  9.94421419,
       10.72114791, 11.1791985 , 11.49984464, 11.75344506, 11.94991061,
       10.15486126, 10.93179498, 11.38984556, 11.71049171, 11.96409212,
       10.54610717, 11.32304088, 11.78109147, 12.10173761, 10.60077969,
       11.37771341, 11.83576399, 10.59644796, 11.37338168, 10.70180523])